# Problem Solving


## 1. 

The practice of introspection would allow the programmer to examine the human cognitive process, studying its working, reasoning and assumptions that allow it to solve problems. Following a train of reasoning is an integral part of translating our sometimes intuitive methods of soving problems that don't require intentional thought and may be missed.  Our brains can fill in missing information or extrapolate from incomplete data without our conscious thought so that we can be efficient. These shortcuts the brain takes makes problem solving easy for us, but a computer model without these shortcuts intentionally programmed would not be able to solve the same problems. 

While it may provide a path to solve problems, it would be hard to closely implement a solution similar to that of the mind because the mind has learned the world and created a schema that may be unique to each person and cannot be examined even with introspection. With this uniqueness comes biases that may be ignored or passed into the model. Introspection would be a good way to model problem solving, but there are some parts of cognition that will be missed even with intense reflection.   
  